# FETCHING DATA FROM NSE

In [68]:
scrip = 'HDFCAMC'

In [69]:
from nsepy import get_history
from datetime import date

stock_df = get_history(symbol='HDFCAMC',
                    start=date(2021,1,1), 
                    end=date(2021,12,31)) 

In [70]:
pd.set_option('display.width', 10000)
print(f"Stock data size:{stock_df.shape}")
display(stock_df.head(2))
display(stock_df.tail(2))

Stock data size:(248, 14)


,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2021-01-01,HDFCAMC,EQ,2918.00,2937.65,3015.0,2935.0,3002.05,2998.05,2990.21,725339,2.168916e+14,49924,208435,0.2874
2021-01-04,HDFCAMC,EQ,2998.05,3015.95,3033.7,2970.4,3024.35,3025.65,3010.37,342044,1.029678e+14,28780,145996,0.4268


,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2021-12-30,HDFCAMC,EQ,2418.80,2420.90,2424.95,2382.0,2410.85,2408.55,2399.93,276518,6.636236e+13,18565,134253,0.4855
2021-12-31,HDFCAMC,EQ,2408.55,2415.95,2449.95,2411.0,2449.00,2446.10,2438.83,117665,2.869652e+13,12815,61668,0.5241


# STOCK ANALYSIS

## CANDLESTICK CHART

In [71]:
import plotly.graph_objects as go

candlestick = go.Candlestick(x=stock_df.index, 
                             open=stock_df['Open'],  
                             high=stock_df['High'], 
                             low=stock_df['Low'], 
                             close=stock_df['Close'])

fig = go.Figure(data=[candlestick])
fig.layout.xaxis.type = 'category' 
fig.show()

# TRADING STRATEGIES/INDICATORS

## MOVING AVERAGE

### SIMPLE MOVING AVERAGE

In [72]:
stock_df['20_sma'] = stock_df['Close'].rolling(window=20).mean()
stock_df[['Close','20_sma']].tail()

,Close,20_sma
Date,,
2021-12-27,2372.20,2466.8525
2021-12-28,2417.90,2462.1825
2021-12-29,2418.80,2455.5625
2021-12-30,2408.55,2448.1550
2021-12-31,2446.10,2441.4925


In [73]:
candlestick1 = go.Candlestick(x=stock_df.index, open=stock_df['Open'], high=stock_df['High'], low=stock_df['Low'], close=stock_df['Close'])
candlestick1





Candlestick({
    'close': array([2998.05, 3025.65, 3091.95, ..., 2418.8 , 2408.55, 2446.1 ]),
    'high': array([3015.  , 3033.7 , 3120.  , ..., 2433.95, 2424.95, 2449.95]),
    'low': array([2935.  , 2970.4 , 2987.95, ..., 2406.05, 2382.  , 2411.  ]),
    'open': array([2937.65, 3015.95, 3000.  , ..., 2415.  , 2420.9 , 2415.95]),
    'x': array([datetime.date(2021, 1, 1), datetime.date(2021, 1, 4),
                datetime.date(2021, 1, 5), ..., datetime.date(2021, 12, 29),
                datetime.date(2021, 12, 30), datetime.date(2021, 12, 31)], dtype=object)
})

### MOVING AVERAGE CHART

In [74]:
# Compute the moving averages for 20 days, 50 days and 200 days
stock_df['20_sma'] = stock_df['Close'].rolling(window=20).mean()
stock_df['50_sma'] = stock_df['Close'].rolling(window=50).mean()
stock_df['200_sma'] = stock_df['Close'].rolling(window=200).mean()

# Plot the moving averages along with the candlestick chart
small_df = stock_df.tail(40)
candlestick = go.Candlestick(x=small_df.index, open=small_df['Open'], high=small_df['High'], low=small_df['Low'], close=small_df['Close'])
sma_20 = go.Scatter(x=small_df.index, y=small_df['20_sma'], name='20 days MA', line={'color': 'blue'})
sma_50 = go.Scatter(x=small_df.index, y=small_df['50_sma'], name='50 days MA', line={'color': 'orange'})
sma_200 = go.Scatter(x=small_df.index, y=small_df['200_sma'], name='200 days MA', line={'color': 'brown'})

# Plot only the last month's data
fig = go.Figure(data=[candlestick,sma_20,sma_50,sma_200],
                layout=go.Layout(title=go.layout.Title(text=f"MOVING AVERAGE CHART - {scrip}")))
fig.layout.xaxis.type = 'category' 
fig.show()


# RSI

In [83]:
import talib
RSI_PERIOD = 14
stock_df['rsi'] = talib.RSI(stock_df['Close'], RSI_PERIOD) 
stock_df['rsi_indicator'] = stock_df['rsi'].apply(lambda x: 'BUY' if x<30 else 'SELL' if x>70 else 'HOLD')
display(stock_df[['Close','rsi','rsi_indicator']].tail(15))

,Close,rsi,rsi_indicator
Date,,,
2021-12-13,2513.90,36.900579,HOLD
2021-12-14,2487.20,34.181544,HOLD
2021-12-15,2456.75,31.344874,HOLD
2021-12-16,2437.75,29.689229,BUY
2021-12-17,2397.95,26.528239,BUY
2021-12-20,2313.60,21.342062,BUY
2021-12-21,2327.15,23.915272,BUY
2021-12-22,2357.55,29.488558,BUY
2021-12-23,2378.30,33.091240,HOLD


# CANDLESTICK PATTERNS

In [90]:
def plot_candle(df, title):
    candlestick = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'])

    # Plot only the last month's data
    fig = go.Figure(data=[candlestick],
                    layout=go.Layout(title=go.layout.Title(text=title)))
    fig.layout.xaxis.type = 'category' 
    fig.show()


## MARUBOZU CANDLE - SINGLE CANDLESTICK PATTERN

In [91]:
# Identify the marubozu candles in the dataset
stock_df['marubozu'] = talib.CDLMARUBOZU(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])

# Subset dataframe for only the marubozu candles
marubozu_candles = stock_df[stock_df['marubozu'] != 0]
display(marubozu_candles[['Close','marubozu']])

# Plot the candlestick chart
plot_candle(marubozu_candles, title = f"Marubozu Candlestick Pattern - {scrip}")


,Close,marubozu
Date,,
2021-03-10,3118.50,-100
2021-03-18,2845.60,-100
2021-04-15,2949.50,100
2021-06-30,2917.45,-100
2021-10-18,2932.45,-100
2021-10-27,2701.25,-100
2021-12-06,2527.95,-100
2021-12-10,2549.75,-100
2021-12-13,2513.90,-100


## DOJI CANDLE

In [96]:
# Identify the marubozu candles in the dataset
stock_df['doji'] = talib.CDLDOJI(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])

# Subset dataframe for only the doji candles
doji_candles = stock_df[stock_df['doji'] != 0]
display(doji_candles[['Open','Close','doji']].tail())


# Plot the candlestick chart
plot_candle(doji_candles, title = f"Doji Candlestick Pattern - {scrip}")


,Open,Close,doji
Date,,,
2021-11-02,2660.4,2665.00,100
2021-11-04,2658.3,2654.95,100
2021-11-09,2670.0,2672.90,100
2021-11-15,2678.0,2680.75,100
2021-12-29,2415.0,2418.80,100
